In [1]:
# CELL 1: SETUP & IMPORTS
import os
import uuid
import time
from datetime import datetime
from dotenv import load_dotenv

# Flask (The Web Server)
from flask import Flask, request, jsonify
from flask_cors import CORS

# AI & Database (The Brain)
from langchain_pinecone import PineconeVectorStore
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_openai import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

# 1. Load API Keys from .env
load_dotenv() 

# Check for keys (Debugging)
if not os.environ.get("OPENAI_API_KEY"):
    print("❌ Error: OPENAI_API_KEY not found. Check your .env file.")
else:
    print("✅ OpenAI Key loaded.")

if not os.environ.get("PINECONE_API_KEY"):
    print("❌ Error: PINECONE_API_KEY not found. Check your .env file.")
else:
    print("✅ Pinecone Key loaded.")

# 2. Initialize Flask App
app = Flask(__name__)
CORS(app) # Allows your React UI to talk to this Python Backend

print("✓ Libraries imported & App initialized.")

c:\Users\teju_\OneDrive\Desktop\AI bot\ai_legal_project\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


✅ OpenAI Key loaded.
✅ Pinecone Key loaded.
✓ Libraries imported & App initialized.


In [2]:
# CELL 2: AI ENGINE SETUP

# --- CONFIGURATION ---
INDEX_NAME = "legal-assistant"  # MUST match your Pinecone Index Name from M1
EMBEDDING_MODEL = "all-MiniLM-L6-v2"

try:
    # 1. Setup Embeddings (The Translator)
    print("⏳ Initializing Embeddings...")
    embeddings = HuggingFaceEmbeddings(model_name=EMBEDDING_MODEL)

    # 2. Connect to Pinecone (The Database)
    print(f"⏳ Connecting to Pinecone Index: {INDEX_NAME}...")
    vectorstore = PineconeVectorStore(
        index_name=INDEX_NAME, 
        embedding=embeddings
    )

    # 3. Setup OpenAI (The Brain)
    llm = ChatOpenAI(
        model_name="gpt-3.5-turbo",
        temperature=0.1 # Low temperature = Factual, Precise Legal Answers
    )

    # 4. Create the Custom Legal Prompt (LegaBot Personality)
    legal_system_prompt = """
    You are LegaBot, a precise legal research assistant. 
    Use the following pieces of context to answer the question at the end.

    GUIDELINES:
    1. GREETINGS & IDENTITY: If the user says "Hi", "Who are you?", or asks general questions about your purpose, respond politely and explain that you help retrieve and analyze legal cases from the database.
    2. CASE COMPARISON: If asked to find differences between cases, use the retrieved context to highlight specific legal points, dates, or outcomes.
    3. Retrieve and list the most relevant statute sections/judgment excerpts.
    4. Provide a concise legal summary (3–5 sentences).
    5. If the answer requires legal interpretation, recommend consulting a lawyer.
    6. If you don't know the answer, say "I don't have enough information in my legal database."
    

    CONTEXT:
    {context}

    QUESTION:
    {question}

    YOUR ANSWER:
    """
    
    PROMPT = PromptTemplate(
        template=legal_system_prompt, 
        input_variables=["context", "question"]
    )

    # 5. Build the Chain (The Full Pipeline)
    qa_chain = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",
        retriever=vectorstore.as_retriever(search_kwargs={"k": 6}), # Get top 6 matching docs
        chain_type_kwargs={"prompt": PROMPT},
        return_source_documents=True
    )
    
    print("✅ AI Engine Ready & Connected to Pinecone.")

except Exception as e:
    print(f"❌ Error initializing AI: {e}")

⏳ Initializing Embeddings...
⏳ Connecting to Pinecone Index: legal-assistant...
✅ AI Engine Ready & Connected to Pinecone.


In [3]:
# CELL 3: API ROUTES WITH SQLITE (PERMANENT STORAGE)

import sqlite3
import json

# --- Database Helper Functions ---

def init_db():
    """Creates the database table if it doesn't exist."""
    conn = sqlite3.connect('chat_history.db')
    c = conn.cursor()
    # Create table to store sessions
    c.execute('''
        CREATE TABLE IF NOT EXISTS sessions (
            session_id TEXT PRIMARY KEY,
            title TEXT,
            last_updated TEXT,
            messages TEXT
        )
    ''')
    conn.commit()
    conn.close()

def save_session(session_id, title, messages):
    """Saves a chat session to the SQLite file."""
    conn = sqlite3.connect('chat_history.db')
    c = conn.cursor()
    # Convert list of messages to JSON string for storage
    messages_json = json.dumps(messages)
    timestamp = str(datetime.now())
    
    c.execute('''
        INSERT OR REPLACE INTO sessions (session_id, title, last_updated, messages)
        VALUES (?, ?, ?, ?)
    ''', (session_id, title, timestamp, messages_json))
    
    conn.commit()
    conn.close()

def load_session(session_id):
    """Loads a specific session."""
    conn = sqlite3.connect('chat_history.db')
    c = conn.cursor()
    c.execute('SELECT title, messages FROM sessions WHERE session_id = ?', (session_id,))
    row = c.fetchone()
    conn.close()
    
    if row:
        return {
            'title': row[0],
            'messages': json.loads(row[1]) # Convert JSON string back to list
        }
    return None

@app.route('/api/sessions/load', methods=['POST'])
def load_chat_session():
    data = request.json

    if not data or not data.get("user_id") or not data.get("session_id"):
        return jsonify({"error": "Unauthorized"}), 401

    session_id = data.get("session_id")
    session_data = load_session(session_id)

    if not session_data:
        return jsonify({"messages": []})

    return jsonify({
        "messages": session_data["messages"]
    })


def get_all_sessions():
    """Gets list of all sessions for the sidebar."""
    conn = sqlite3.connect('chat_history.db')
    c = conn.cursor()
    c.execute('SELECT session_id, title, last_updated FROM sessions ORDER BY last_updated DESC')
    rows = c.fetchall()
    conn.close()
    
    sessions_list = []
    for row in rows:
        sessions_list.append({
            'id': row[0],
            'title': row[1],
            'last_updated': row[2]
        })
    return sessions_list

# Initialize DB when this cell runs
init_db()
print("✓ SQLite Database Connected & Ready.")


# --- AI Logic (Same as before) ---

def generate_ai_response(user_query):
    try:
        result = qa_chain.invoke({"query": user_query})
        return result["result"]
    except Exception as e:
        print(f"Error: {e}")
        return "I apologize, but I encountered an error."

def create_session_title(user_message):
    return user_message[:30] + "..." if len(user_message) > 30 else user_message


# --- Routes (Updated to use SQLite functions) ---

@app.route('/api/sessions', methods=['POST'])
def get_sessions():
    data = request.json
    if not data or not data.get("user_id"):
        return jsonify({"error": "Please login first"}), 401

    return jsonify(get_all_sessions())

@app.route('/api/sessions/<session_id>', methods=['GET'])
def get_single_session(session_id):
    data = load_session(session_id)
    if data:
        return jsonify(data)
    return jsonify({"error": "Session not found"}), 404

@app.route('/api/chat', methods=['POST'])
def chat():
    data = request.json
    if not data.get("user_id"):
        return jsonify({"error": "Please login first"}), 401
    user_message = data.get('message')
    session_id = data.get('session_id')

    if not user_message:
        return jsonify({"error": "Message is required"}), 400

    # 1. Load existing history or start new
    current_history = []
    title = create_session_title(user_message)
    
    if session_id:
        existing_data = load_session(session_id)
        if existing_data:
            current_history = existing_data['messages']
            title = existing_data['title'] # Keep old title
        else:
            # Session ID sent but not in DB (rare edge case), treat as new
            pass 
    else:
        session_id = str(uuid.uuid4())

    # 2. Add User Message
    current_history.append({
        'sender': 'user',
        'text': user_message,
        'timestamp': str(datetime.now())
    })

    # 3. Generate Response
    ai_response_text = generate_ai_response(user_message)

    # 4. Add AI Message
    current_history.append({
        'sender': 'bot',
        'text': ai_response_text,
        'timestamp': str(datetime.now())
    })

    # 5. Save everything to SQLite
    save_session(session_id, title, current_history)

    return jsonify({
        "session_id": session_id,
        "bot_response": ai_response_text,
        "history": current_history
    })

print("✓ API Routes Updated for Permanent Storage.")

✓ SQLite Database Connected & Ready.
✓ API Routes Updated for Permanent Storage.


In [4]:
# CELL 4: USERS TABLE (AUTH)

def init_users_db():
    conn = sqlite3.connect('chat_history.db')
    c = conn.cursor()

    c.execute('''
        CREATE TABLE IF NOT EXISTS users (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            full_name TEXT,
            email TEXT UNIQUE,
            password_hash TEXT,
            created_at TEXT
        )
    ''')

    conn.commit()
    conn.close()

init_users_db()
print("✓ Users table ready.")


✓ Users table ready.


In [5]:
from werkzeug.security import generate_password_hash, check_password_hash

# CELL 5: SIGNUP API
@app.route('/api/signup', methods=['POST'])
def signup():
    data = request.json

    full_name = data.get('full_name')
    email = data.get('email')
    password = data.get('password')

    if not email or not password:
        return jsonify({"error": "Email and password required"}), 400

    password_hash = generate_password_hash(password)

    try:
        conn = sqlite3.connect('chat_history.db')
        c = conn.cursor()
        c.execute('''
            INSERT INTO users (full_name, email, password_hash, created_at)
            VALUES (?, ?, ?, ?)
        ''', (full_name, email, password_hash, str(datetime.now())))
        conn.commit()
        conn.close()

        return jsonify({"message": "Signup successful"}), 201

    except sqlite3.IntegrityError:
        return jsonify({"error": "Email already exists"}), 409


In [6]:
# CELL 6: LOGIN API
@app.route('/api/login', methods=['POST'])
def login():
    data = request.json
    email = data.get('email')
    password = data.get('password')

    conn = sqlite3.connect('chat_history.db')
    c = conn.cursor()
    c.execute('SELECT id, password_hash FROM users WHERE email = ?', (email,))
    user = c.fetchone()
    conn.close()

    if not user or not check_password_hash(user[1], password):
        return jsonify({"error": "Invalid credentials"}), 401

    return jsonify({
        "user_id": user[0],
        "message": "Login successful"
    })

# SIMPLE AUTH CHECK HELPER
def is_authenticated(data):
    return bool(data.get("user_id"))


In [7]:
# CELL 7: LOGOUT API

@app.route('/api/logout', methods=['POST'])
def logout():
    return jsonify({"message": "Logged out"})

In [8]:
# CELL 8: RUN SERVER

if __name__ == '__main__': 
    print("🚀 Server running on http://localhost:5000") 
    app.run(port=5000)

🚀 Server running on http://localhost:5000
 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [22/Dec/2025 21:48:24] "OPTIONS /api/sessions HTTP/1.1" 200 -
127.0.0.1 - - [22/Dec/2025 21:48:24] "OPTIONS /api/sessions HTTP/1.1" 200 -
127.0.0.1 - - [22/Dec/2025 21:48:24] "POST /api/sessions HTTP/1.1" 200 -
127.0.0.1 - - [22/Dec/2025 21:48:25] "POST /api/sessions HTTP/1.1" 200 -
127.0.0.1 - - [22/Dec/2025 21:48:26] "OPTIONS /api/sessions/load HTTP/1.1" 200 -
127.0.0.1 - - [22/Dec/2025 21:48:26] "POST /api/sessions/load HTTP/1.1" 200 -
127.0.0.1 - - [22/Dec/2025 21:48:28] "POST /api/sessions/load HTTP/1.1" 200 -
127.0.0.1 - - [22/Dec/2025 21:48:31] "OPTIONS /api/sessions/load HTTP/1.1" 200 -
127.0.0.1 - - [22/Dec/2025 21:48:32] "POST /api/sessions/load HTTP/1.1" 200 -
127.0.0.1 - - [22/Dec/2025 21:48:35] "POST /api/sessions/load HTTP/1.1" 200 -
